In [1]:

import numpy as np
import pandas as pd
print(np.__version__)
print(pd.__version__)
from sklearn.metrics.pairwise import cosine_similarity

1.15.4
0.23.4


In [2]:

def mergeDatasets(path):	#takes the path and returns a merged dataset
    data_m = pd.read_csv(path+'movies.csv')
    data_r = pd.read_csv(path+'ratings.csv')
    data_u = pd.read_csv(path+'users.csv')

    movie_ratings = pd.merge(data_m, data_r)
    lens = pd.merge(movie_ratings, data_u)
    #print(lens.head())

    #most_rated = lens.groupby('title').size().sort_values(ascending=False)[:25]
    #print(most_rated)

    return lens

def createUserItemPair(data):
    ## picks top k rated movies & top k users with most ratings
    ## selects subset containing only above users and movies
    ## creates (user, item) pair matrix with rating as cell value
    lens = data.copy()

    most_rated_users = lens.groupby('userId').size().sort_values(ascending=False)
    mru_keys = list(most_rated_users.keys())

    most_rated_movies = lens.groupby('itemId').size().sort_values(ascending=False)
    mrm_keys = list(most_rated_movies.keys())


    data = data.loc[data['userId'].isin(mru_keys) & data['itemId'].isin(mrm_keys)]
    data = data.pivot(index='userId', columns='itemId', values='rating')
    data = np.where(np.isnan(data),0, data)
    #print(data.head())
    return data

def createMatrixI(data):
    I = data.copy()
    I = np.asarray(I)
    I = np.where(np.isnan(I), 0, 1)
    
    return I    


In [3]:
def user_user_similarity_matrix(data):
    uu_matrix = cosine_similarity(data)
    return uu_matrix
    
def prediction_matrix(A, uu_matrix):
    pred_matrix = np.zeros(A.shape)
    K = A.shape[0]
    w = A.shape[1]
    mean_ratings_of_users = []
    for i in range(K):
        mean_ratings_of_users.append(np.mean(A[i]))
    print(mean_ratings_of_users)
    return None
    for i in range(0, w):
        mean_i = np.mean(A[i])
        '''pred_matrix[i] = mean_i + np.sum(np.subtract(A[i], np.full((K,), mean_i)) * uu_matrix[i])
                            / np.sum(uu_matrix[i]) '''
        for a in range(0, K):
            pred_matrix[a][i] = mean_i + np.sum
        
    
def collab_filtering():
    pass

In [4]:
def main():
    path = 'data/'
    data = mergeDatasets(path)

    A = createUserItemPair(data)
    I = createMatrixI(A)
    uu_matrix = user_user_similarity_matrix(A)
    pred_matrix = prediction_matrix(A, uu_matrix)
    print(uu_matrix)
    print(uu_matrix.shape)


if __name__ == '__main__':
    main()

[0.05990286022665947, 0.12924986508364814, 0.05369670804101457, 0.023745277927684834, 0.1681057744198597, 0.0747436589314625, 0.03615758229897464, 0.1457096600107933, 0.10685375067458176, 0.4452239611440907, 0.1211548839719374, 0.023745277927684834, 0.09875876956287102, 0.02239611440906638, 0.1802482460874258, 0.028602266594711278, 0.23205612520237454, 0.30032379924446845, 0.2458175930922828, 0.02644360496492175, 0.017269293038316244, 0.2458175930922828, 0.27199136535348084, 0.14490016189962224, 0.08580679978413383, 0.3194819212088505, 0.07879114948731786, 0.1084727468969239, 0.10442525634106854, 0.040474905558553695, 0.11980572045331894, 0.04695089044792229, 0.3691311386940097, 0.17107393416082028, 0.1891527253103076, 0.397733405288721, 0.0528872099298435, 0.09660010793308149, 0.05963302752293578, 0.08931462493254183, 0.023475445223961146, 0.2331354560172693, 0.02671343766864544, 0.1891527253103076, 0.23610361575822988, 0.0466810577441986, 0.023205612520237454, 0.49514301133297356, 0.